In [1]:
import os
from os.path import isfile, join

import numpy as np

import imageio as img
from tqdm import tqdm

from pprint import pprint
import plotly as py
import plotly.graph_objects as go

if not os.path.exists('images'):
    os.mkdir('images')
    
images_dir = os.getcwd() + '\\images'
print(f'Saving imaged to dir: {images_dir}')

os.chdir('../')
if not os.path.exists('Visualisations'):
    os.mkdir('Visualisations')

visualisations_dir = os.getcwd() + '\\Visualisations'

from Logistic_Map_Generator import LogisticMapGenerator

Saving imaged to dir: C:\Users\subst\Projects\GitBase\Logistic-Map-Generator\Testing\images


In [2]:
def get_labels(axis_num, tick_counts):
    increments = np.linspace(0.0, 1.0, num=tick_counts+1, endpoint=True)\
                 if axis_num != 1\
                 else np.linspace(0.0, 1.0, num=tick_counts+1, endpoint=True)[::-1]
    labels = np.array([[increments[idx],increments[-(1+idx)]]
                     for idx in range(tick_counts+1)
                    ]
                   )
    labels = np.round(np.insert(labels, axis_num, 0.0, axis=1),1)[:-1]
    labels = [str(tuple(label)) for label in labels]
    return labels

def make_axis(title, axis_num, tick_angle, tick_counts):
    return {'title': title,
            'titlefont': { 'size': 10 },
            'tickangle': tick_angle,
            'tickfont': { 'size': 6 },
            'tickcolor': 'rgba(0,0,0,0)',
            'tickmode' : 'array',
            'tickvals': [idx/tick_counts for idx in range(tick_counts)],
            'ticktext': get_labels(axis_num=axis_num, tick_counts=tick_counts),
            'ticklen': 50,
            'showline': True,
            'showgrid': True
           }

In [5]:
keys = ['x @ t = 0', 'x @ t = +1', 'x @ t = +2']
total_itts = 333
for idx, r_val in tqdm(enumerate(np.linspace(start=3.0, stop=4.0, num=total_itts, endpoint=False)), total=total_itts):
    
    # generate new map with random start (x_0, and new r_val)
    x=np.random.rand()
    map_gen = LogisticMapGenerator(x=x, r=r_val, alphabet='ABCD', depth=6, ret_type='ternary', ret_history=3)
    
    # Extract data for 50000 points
    # TODO: store data in a dictionary to be able to look up periodicity (key = ternary values)
    rawData = [{key:val for key,val in zip(keys, next(map_gen))} for _ in range(50000)]
    
    # Create Figure
    fig = go.Figure(go.Scatterternary({'mode': 'markers',
                                       'a': [i for i in map(lambda x: x['x @ t = 0'], rawData)],
                                       'b': [i for i in map(lambda x: x['x @ t = +1'], rawData)],
                                       'c': [i for i in map(lambda x: x['x @ t = +2'], rawData)],
                                       'marker': {'symbol': 100,
                                                  'color': '#DB7365',
                                                  'size':4,
                                                  'line': { 'width': 1 },
                                                  'opacity' : 0.625
                                                 }
                                      }
                                     )
                   )

    # Add Equilateral Triangle Centroid -> Ternary [1/3, 1/3, 1/3] point
    fig.add_trace(go.Scatterternary({'mode': 'markers',
                                      'a': [1/3.],
                                      'b': [1/3.],
                                      'c': [1/3.],
                                      'marker': {'symbol': 'triangle-up-open',
                                                 'color': '#DB7365',
                                                 'size':10,
                                                 'line': { 'width': 2 }
                                                }
                                    }
                                   )
                 )

    fig.update_layout({'ternary': {'sum': 1,
                                   'aaxis': make_axis(title='x@ t = 0', axis_num=2, tick_angle=60, tick_counts=10),
                                   'baxis': make_axis(title='x@<br>t = +1', axis_num=0, tick_angle=-60, tick_counts=10),
                                   'caxis': make_axis(title='x@<br>t = +2', axis_num=1, tick_angle=0, tick_counts=10)
                                  },
                       'annotations': [{'showarrow': False,
                                        'text': f'Ternary Plot f([t=0, t=+1, t=+2]) of Logistic Map for <br>r = {r_val:.4f}'
                                        'x': 0.5,
                                        'y': 0.98,
                                        'font': { 'size': 14 }
                                       }],
                       'autosize':False,
                       'showlegend':False,
                       'width':768,
                       'height':768,
                       'margin':go.layout.Margin(l=80,
                                                 r=80,
                                                 b=0,
                                                 t=0,
                                                 pad=0
                                                ),
                       'paper_bgcolor':'rgba(0.95,0.95,0.95,1.0)',
                       'plot_bgcolor':'rgba(0.95,0.95,0.95,1.0)'
                      }
                     )

    # Save File
    r_val_str = str(round(r_val, 5)).replace('.', '_')
    fig.write_image(images_dir + f'\\{idx} - r_value = {r_val_str}.jpeg')

SyntaxError: invalid syntax (<ipython-input-5-7ef227094ac4>, line 50)

In [4]:
# Get file_names
file_names = [file_name for file_name in os.listdir(images_dir)
              if isfile(join(images_dir, file_name))
             ]
# Sort file_names (by frame number) to guarantee order
file_names = {int(file_name.split(' - ')[0]): join(images_dir, file_name)
              for file_name in file_names
             }
# get ordered file_names
file_names = [file_names[idx] for idx in range(len(file_names))]

#set output file (.gif)
output_file = visualisations_dir + '\\Ternary Plot [t=0, t=+1, t=+2] of Logistic Map.gif'

# Create gif with imageio
with img.get_writer(output_file, mode='I') as writer:
    for file_name in file_names:
        image = img.imread(file_name)
        writer.append_data(image)